In [1]:
# Import Pyomo and the required modules
from pyomo.environ import *
from sklearn import metrics
import warnings
import pandas as pd
import numpy as np
from echo.Data import *
import os
import seaborn as sns

In [2]:
data=Spotprice(2021,'QLD1',60)
data

C:\Users\Administrator\ECHO model\echo\Data.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = pd.to_datetime(df['Time'])


,Time,Prices
0,2021-01-01 00:00:00,0.037943
1,2021-01-01 01:00:00,0.034325
2,2021-01-01 02:00:00,0.028836
3,2021-01-01 03:00:00,0.026916
4,2021-01-01 04:00:00,0.030528
...,...,...
8755,2021-12-31 19:00:00,0.183745
8756,2021-12-31 20:00:00,0.118750
8757,2021-12-31 21:00:00,0.096695
8758,2021-12-31 22:00:00,0.126263


In [3]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [4]:
file_path=r'D:\Do it\Phd\ECHO\ECHO\Dataframe_QLD.csv'
source=pd.read_csv(file_path, index_col=0)
source

,Time,Prices,Solar,Carbon intensity,Wind,mean,std
0,0:00,37.942625,0.000000,0.881917,46797.53376,50.280578,11.987485
1,1:00,34.325000,0.000000,0.871882,16961.54208,44.890720,10.028155
2,2:00,28.835833,0.000000,0.905274,19904.54176,40.268061,8.492114
3,3:00,26.915833,0.000000,0.907171,14269.79456,39.055020,8.064904
4,4:00,30.527903,0.000000,0.901070,5877.58912,42.049695,8.513659
...,...,...,...,...,...,...,...
8755,19:00,183.745098,8.279688,0.438912,37649.41856,84.526292,37.933099
8756,20:00,118.750448,0.000000,0.573657,33310.64160,74.446324,26.715071
8757,21:00,96.694904,0.000000,0.729462,22628.78048,61.804848,20.137540
8758,22:00,126.262897,0.000000,0.520182,7187.47552,62.074745,18.603433


In [53]:
def ECHO(year,location,step,grid,supply_ratio,source_df,num_interval,ratio,SO_k,batch_interval):
    
    #data import
    data=Spotprice(year,location,step)
    data['Carbon intensity']=carbon_intensity(year,location,step)['Intensity_Index']
    data['Mean Carbon intensity']=Mean_carbon_intensity(year,location,step)['Intensity_Index']
    new_solar=divide(source_df['Solar'],step)
    new_wind=divide(source_df['Wind'],step)
    data['Solar']=new_solar
    data['Wind']=new_wind
    source_df=data       
    end_index=60/step*8759

    # Initialize the data
      
    m = ConcreteModel()
    num_simulation = len(source_df)
    #set the simultaneity obligation interval
    interval=num_interval
    m.time_periods = RangeSet(0,num_simulation-1)
    m.check_periods=RangeSet(0,num_simulation-1,interval)

    m.interval=RangeSet(0,num_simulation)
    m.batch_interval=RangeSet((batch_interval-1)*60/step,num_simulation-1,batch_interval*60/step)       #using different batch intervals
    number_batch_interval = len(list(m.batch_interval))
    #please note that The minimum unit of batch_interval is 1 hour.
    print("Number of batch intervals:",number_batch_interval)
    #initial value of cost
    m.c_pv = Param(initialize=1122.7)
    m.c_wind =Param(initialize=1455) 
    m.c_el = Param(initialize=1067)
    m.c_hydrogen_storage = Param(initialize=17.66)

    m.CRF =Param(initialize= 0.07822671821)
    m.pv_FOM = Param(initialize=12.7)
    m.wind_FOM = Param(initialize=18.65)
    m.el_FOM =Param(initialize=37.4) 
    m.el_VOM = Param(initialize=0.075)

    #m.max_power_export =Param(initialize=175e3)
    #Transmission cost may varied according to the scale
    if grid==1:
        print("On-Grid")
        m.max_power_export=Var(domain=NonNegativeReals)
    else:
        print("Off-Grid")
        m.max_power_export=Param(initialize=0)
    m.grid_connection_fee = Var(domain=NonNegativeReals)             #39552000    

    
    #m.electrolyser_max_input = Param(initialize=175) # kW max input
    #m.initial_h2_storage_value = 749752 ###749792
    m.pv_ref_size =Param(initialize=1000)
    m.wind_ref_size =Param(initialize=320000)
    
    
    #renewable generation
    m.pv_ref_generation = Param(m.time_periods, initialize={i:  float(source_df.loc[i, 'Solar']) for i in m.time_periods})
    m.wind_ref_generation=Param(m.time_periods, initialize={i:  float(source_df.loc[i, 'Wind']) for i in m.time_periods})
    
    #price and carbon intensity
    m.price = Param(m.time_periods, initialize={i:  float(source_df.loc[i, 'Prices']) for i in m.time_periods})
    m.carbon_intensity=Param(m.time_periods, initialize={i:  float(source_df.loc[i, 'Carbon intensity']) for i in m.time_periods})
    m.mean_carbon_intensity=Param(m.time_periods, initialize={i:  float(source_df.loc[i, 'Mean Carbon intensity']) for i in m.time_periods})
    
    #load
    #initial_values={i: 18000*8760 / 60 * step /number_batch_interval if i in m.batch_interval else 0 for i in m.time_periods}   #Test the batch_interval
    initial_values={i: 18000/60*step  for i in m.time_periods}        #Test model validation
    m.Load=Param(m.time_periods, initialize=initial_values)
    
    m.production_amount=Var()
    #check whether the flow can be active or not using Big M method
    m.is_grid_pin_active = Var(m.time_periods, within=Binary)
    m.is_grid_pout_active = Var(m.time_periods, within=Binary)
    m.M=Param(initialize=1e10)
    

    
#######Define variables

    #Variable capacity
    #m.pv_capacity=Var(domain=NonNegativeReals) 
    #m.wind_capacity=Var(domain=NonNegativeReals) 
    #m.h2_storage_capacity=Var(domain=NonNegativeReals)
    #m.electrolyser_max_input=Var(domain=NonNegativeReals)
    
    #Fixed capacity
    m.pv_capacity=Param(initialize=1455444*ratio)
    m.wind_capacity=Param(initialize=2015089*ratio)
    m.h2_storage_capacity = Param(initialize=10003211)
    m.electrolyser_max_input = Param(initialize=1750162)        #175kw
    
    
    m.pv_pout = Var(m.time_periods, domain=NonPositiveReals)  #power out of PV plant (kW)
    m.wind_pout = Var(m.time_periods, domain=NonPositiveReals)  #power out of wind farm (kW)
    m.curtail_wind = Var(m.time_periods, within=NonPositiveReals)  #curtailed power (kW)
    m.curtail_solar = Var(m.time_periods, within=NonPositiveReals)  #curtailed power (kW)
    
    m.CP_wind = Var(m.time_periods, domain=NonNegativeReals)  
    m.CP_pv= Var(m.time_periods, domain=NonNegativeReals) 
    
    #Ports between grid and connection point
    m.grid_pout= Var(m.time_periods, domain=NonPositiveReals)
    m.grid_pin=Var(m.time_periods, domain=NonNegativeReals)
    m.CP_grid_out= Var(m.time_periods, domain=NonPositiveReals)
    m.CP_grid_in= Var(m.time_periods, domain=NonNegativeReals)

    
    m.CP_el=Var(m.time_periods, domain=NonPositiveReals)
    m.el_pin=Var(m.time_periods, domain=NonNegativeReals)
    
    m.el_pout=Var(m.time_periods, domain=NonPositiveReals)
    
    #H2 connection point node
    m.H2CP_el= Var(m.time_periods, domain=NonNegativeReals) 
    m.H2CP_h2_storage= Var(m.time_periods, domain=Reals)     
    m.H2CP_h2_demand= Var(m.time_periods, domain=NonPositiveReals)       #export will be negative value
    #H2 storage node
    m.h2_storage_p=Var(m.time_periods, domain=Reals) 
    m.h2_storage_level=Var(m.interval, domain=NonNegativeReals) 
    m.initial_h2_storage_value=Var(domain=NonNegativeReals)
    #m.production_amount=Var(domain=NonNegativeReals)
    
    m.CO2=Var(m.time_periods, domain=Reals) 
    m.Mean_CO2=Var(m.time_periods, domain=Reals) 
    
    m.grid_cost=Var()
    m.capex=Var()
    m.LCOH=Var()
    #m.sell_profit=Var(m.time_periods, domain=Reals)
    #m.purchase_cost=Var(m.time_periods, domain=Reals)

    
    
####### Define the constraints 

    if SO_k==1:
        if num_interval==0:
            print("Simultaneity_obligation is off")
        else:
            print("Simultaneity_obligation is on")
            def simultaneity_rule(m, j):
                return sum(m.grid_pin[i] + m.grid_pout[i] for i in range(j, min(j + interval, 8760))) >= 0
            m.simultaneity_constraint = Constraint(m.check_periods, rule=simultaneity_rule)
    if SO_k==0:
        num_interval=0
        print("Simultaneity_obligation is off")
    
    #grid_connection_fee is related to the scale     155 unit USD/kW
    m.con_grid_connection_fee=Constraint(expr=m.max_power_export*155==m.grid_connection_fee)
    #m.con_max_power_export=Constraint(expr=m.max_power_export<=175000)
    
    
    def constraint_rule_el_pin(m, i):       
        return  m.el_pin[i]<=m.electrolyser_max_input*step/60       
    m.con_el_pin= Constraint(m.time_periods, rule=constraint_rule_el_pin)
    
    def constraint_rule_H2storage_capacity(m, i):
        return  m.h2_storage_level[i]<=m.h2_storage_capacity
    m.con_H2storage_capacity= Constraint(m.time_periods, rule=constraint_rule_H2storage_capacity)
   
    def constraint_rule_grid_buy_max(m, i):
        return  m.grid_pout[i]>=-m.max_power_export
    m.con_grid_buy_max= Constraint(m.time_periods, rule=constraint_rule_grid_buy_max)
    
    def constraint_rule_grid_sell_max(m, i):
        return  m.grid_pin[i]<=m.max_power_export
    m.con_grid_sell_max= Constraint(m.time_periods, rule=constraint_rule_grid_sell_max)
  
    m.con_hydrogen_storage=Constraint(expr=m.h2_storage_level[end_index]==m.initial_h2_storage_value)     
    #m.con_production_amount=Constraint(expr=m.production_amount==18000*num_simulation-m.initial_h2_storage_value)
    
    m.con_cost=Constraint(expr=m.grid_cost==sum(-m.grid_pout[i]*(m.price[i]+0.01)-(m.grid_pin[i]*(m.price[i]-0.041)) for i in m.time_periods))   
    #0.034 means the integration cost of equipment which can help newly installed renewable energy reliable
    def constraint_rule_CO2(m, i):
        return  m.CO2[i]==-(m.grid_pout[i]+m.grid_pin[i])*m.carbon_intensity[i]      #According to Energy interaction
    m.con_CO2= Constraint(m.time_periods, rule=constraint_rule_CO2)
    
    def constraint_rule_Mean_CO2(m, i):
        return  m.Mean_CO2[i]==-(m.grid_pout[i]+m.grid_pin[i])*m.mean_carbon_intensity[i]
    m.con_Mean_CO2= Constraint(m.time_periods, rule=constraint_rule_Mean_CO2)
 
    #test the indirect grid CO2 = 0
    #m.con_CO2_0=Constraint(expr=sum(m.CO2[i] for i in m.time_periods)==0 )

    #Renewable generation
    #Ensure the major part of energy supply into the electrolyser, the renewable electricity into the electrolyser must exceed certain proportion  

    if supply_ratio != 0:
        print('supply_ratio:',supply_ratio)
        m.con_supply=Constraint(expr=sum(m.CP_wind[i] + m.CP_pv[i] for i in m.time_periods)*supply_ratio >=sum(m.grid_pin[i] for i in m.time_periods))      
    else:
        print('supply_ratio:',supply_ratio,'No restriction on selling amount')
        
    m.con_production=Constraint(expr=sum(m.Load[i] for i in m.time_periods)==m.production_amount)
        
##############PV################
    def constraint_rule_pv(m, i):
        return  m.pv_pout[i]+ m.curtail_solar[i]==-1*m.pv_capacity/m.pv_ref_size*m.pv_ref_generation[i]
    m.con_pv= Constraint(m.time_periods, rule=constraint_rule_pv)

##############Wind################ 
    def constraint_rule_wind(m, i):
        return  m.wind_pout[i]+ m.curtail_wind[i]==-1*m.wind_capacity/m.wind_ref_size*m.wind_ref_generation[i]
    m.con_wind = Constraint(m.time_periods, rule=constraint_rule_wind)
    
##############Grid################
    def constraint_rule_CP_grid_1(m, i):
        return  m.grid_pout[i]+m.CP_grid_in[i]==0
    m.con_grid1 = Constraint(m.time_periods, rule=constraint_rule_CP_grid_1)
        
    def constraint_rule_CP_grid_2(m, i):
        return  m.grid_pin[i]+m.CP_grid_out[i]==0
    m.con_grid2 = Constraint(m.time_periods, rule=constraint_rule_CP_grid_2)
    
    def constraint_rule_CP_grid(m, i):
        return m.grid_pin[i] <= m.is_grid_pin_active[i] * m.M  # M is a large constant
    m.con_grid_pin = Constraint(m.time_periods, rule=constraint_rule_CP_grid)

    def constraint_rule_CP_grid(m, i):
        return -m.grid_pout[i] <= m.is_grid_pout_active[i] * m.M  # M is a large constant
    m.con_grid_pout = Constraint(m.time_periods, rule=constraint_rule_CP_grid)

    def constraint_rule_CP_grid_one_flow(m, i):
        return m.is_grid_pin_active[i] + m.is_grid_pout_active[i] == 1
    m.con_grid_one_flow = Constraint(m.time_periods, rule=constraint_rule_CP_grid_one_flow)

    
    def constraint_rule_CP_wind(m, i):
        return  m.wind_pout[i]+m.CP_wind[i]==0
    m.con_CP_wind = Constraint(m.time_periods, rule=constraint_rule_CP_wind)
    
    def constraint_rule_CP_pv(m, i):
        return  m.pv_pout[i]+m.CP_pv[i]==0
    m.con_CP_pv = Constraint(m.time_periods, rule=constraint_rule_CP_pv)
    
    def constraint_rule_CP_el(m, i):
        return  m.el_pin[i]+m.CP_el[i]==0
    m.con_CP_el = Constraint(m.time_periods, rule=constraint_rule_CP_el)
    
    def constraint_rule_el(m, i):      #this indicates the power balance in different time steps for example if step is 15min the power consumed is 1/4 kWh
        return  m.el_pin[i]*0.7*(1/(39.4+0.83+0.41))+m.el_pout[i]==0
    m.con_el = Constraint(m.time_periods, rule=constraint_rule_el)
#electrical connection point
    def constraint_rule_CP(m, i):
        return  m.CP_el[i]+m.CP_pv[i]+m.CP_wind[i]+m.CP_grid_out[i]+m.CP_grid_in[i]==0
    m.con_CP = Constraint(m.time_periods, rule=constraint_rule_CP)

#H2 connection point
    def constraint_rule_H2CP(m, i):
        return  m.H2CP_el[i]+m.H2CP_h2_storage[i]+m.H2CP_h2_demand[i]==0
    m.con_H2CP = Constraint(m.time_periods, rule=constraint_rule_H2CP)
    
    def constraint_rule_H2CP_el(m, i):
        return  m.el_pout[i]+m.H2CP_el[i]==0
    m.con_H2CP_el = Constraint(m.time_periods, rule=constraint_rule_H2CP_el)
    
    def constraint_rule_H2CP_H2storage(m, i):
        return  m.H2CP_h2_storage[i]+ m.h2_storage_p[i]==0
    m.con_H2CP_H2storage= Constraint(m.time_periods, rule=constraint_rule_H2CP_H2storage)
    
    
    #def constraint_rule_H2storage(m,i):
     #   return  m.h2_storage_level[i]==m.h2_storage_level[0]+sum(m.h2_storage_p[i])
    #m.con_H2storage_level= Constraint(m.time_periods, rule=constraint_rule_H2storage)
    def constraint_rule_H2storage(m, i):
        if i == 0:
            return m.h2_storage_level[i] == m.initial_h2_storage_value+m.h2_storage_p[i]
        else:
            return m.h2_storage_level[i] == m.h2_storage_level[i - 1] + m.h2_storage_p[i]

# Add the constraint
    m.con_H2storage_level = Constraint(m.time_periods, rule=constraint_rule_H2storage)
    
    
    def constraint_rule_H2storage_level(m,i):
        return  m.h2_storage_level[i]>=0
    m.con_H2storage_level1= Constraint(m.time_periods, rule=constraint_rule_H2storage_level)
    
    #def constraint_rule_H2storage_initial(m):
    #    return m.h2_storage_level[0] == m.initial_h2_storage_value
# Add the initial constraint to the model
   # m.con_H2storage_initial = Constraint(rule=constraint_rule_H2storage_initial)
    
    def constraint_rule_H2CP_H2demand(m, i):
        return  m.H2CP_h2_demand[i]+ m.Load[i]==0
    m.con_H2CP_H2demand= Constraint(m.time_periods, rule=constraint_rule_H2CP_H2demand)
    
    #Carbon emission constraint:
    #m.con_MEF_limit=Constraint(expr=sum(m.CO2[i] for i in m.time_periods)<=0)
    #m.con_Mean_limit=Constraint(expr=sum(m.Mean_CO2[i] for i in m.time_periods)<=0)

    
    
    # LCOH and capex
    m.con_capex=Constraint(expr=m.capex==m.c_pv*m.pv_capacity+
                      m.c_wind*m.wind_capacity+
                      m.c_el*m.electrolyser_max_input+
                      m.c_hydrogen_storage*m.h2_storage_capacity
                      +m.grid_connection_fee)
    #Capex limitation:
    #m.con_capex_limit=Constraint(expr=m.capex<=4.2*1e6)
    
    
    def LCOH_constraint(m,i):
        return m.LCOH == (
            (m.capex * m.CRF + m.pv_capacity * m.pv_FOM + m.wind_capacity * m.wind_FOM + m.electrolyser_max_input * m.el_FOM+m.grid_cost/1.49))
            #/m.production_amount
            #+ m.el_VOM)
    m.LCOH_constraint = Constraint(rule=LCOH_constraint)
    
    m.obj = Objective(expr=m.LCOH, sense=minimize)
        # Solve the linear programming problem
    #solver = SolverFactory('gurobi')  # Use GLPK solver, you can install it separately
    solver = SolverFactory('gurobi')
    #solver = SolverFactory('ipopt')  # Use a nonlinear solver, e.g., IPOPT

    solver.options['MIPFocus'] = 3
    solver.options['MIPGap'] = 1e-6  # Set the MIP gap tolerance to control the precision
    solver.options['FeasibilityTol'] = 1e-9 
    results = solver.solve(m)
    
# Display the results
    if results.solver.termination_condition == TerminationCondition.optimal:
        print("Optimal Solution Found")
        variable_values = {}
#print the flow value
        CP_grid_out=list()
        CP_grid_in=list()
        grid_interaction=list()
        grid_pout=list()
        grid_pin=list()
        CP_wind=list()
        CP_pv=list()
        CP_el=list()
        pv_pout=list()
        wind_pout=list()
        curtail_wind=list()
        curtail_solar=list()
        el_pin=list()
        el_pout=list()
        h2_storage_p=list()
        h2_storage_level=list()
        price=list()
        CO2=list()
        Mean_CO2=list()
        load=list()
        
        
        for Time in m.time_periods:
            CP_grid_out.append(value(m.CP_grid_out[Time]))
            CP_grid_in.append(value(m.CP_grid_in[Time]))
            grid_interaction.append(value(m.grid_pout[Time])+value(m.grid_pin[Time]))
            grid_pout.append(value(m.grid_pout[Time]))
            grid_pin.append(value(m.grid_pin[Time]))
            pv_pout.append(value(m.pv_pout[Time]))
            wind_pout.append(value(m.wind_pout[Time]))
            curtail_wind.append(value(m.curtail_wind[Time]))
            curtail_solar.append(value(m.curtail_solar[Time]))
            CP_wind.append(value(m.CP_wind[Time]))
            CP_pv.append(value(m.CP_pv[Time]))
            CP_el.append(value(m.CP_el[Time]))
            el_pin.append(value(m.el_pin[Time]))
            el_pout.append(value(m.el_pout[Time]))
            h2_storage_p.append(value(m.h2_storage_p[Time]))
            h2_storage_level.append(value(m.h2_storage_level[Time]))
            price.append(value(m.price[Time]))
            CO2.append(value(m.CO2[Time]))
            Mean_CO2.append(value(m.Mean_CO2[Time]))
            load.append(value(m.Load[Time]))
        #for time_interval in m.check_periods:
        #    print("Time point",time_interval,":",sum(value(m.grid_pin[i]) + value(m.grid_pout[i]) for i in range(time_interval, min(time_interval + interval, 8760))))
               
        data = {'grid_interaction':grid_interaction,
                'CP_grid_out': CP_grid_out,
                'CP_grid_in': CP_grid_in,
                'grid_pout':grid_pout,
                'grid_pin':grid_pin,
                'pv_pout': pv_pout,
                'wind_pout': wind_pout,
                'curtail_wind':curtail_wind,
                'curtail_solar':curtail_solar,
                'CP_wind': CP_wind,
                'CP_pv': CP_pv,
                'CP_el': CP_el,
                'el_pin': el_pin,
                'el_pout': el_pout,
                'h2_storage_p': h2_storage_p,
                'h2_storage_level': h2_storage_level,
                'price':price,
                'CO2':CO2,
                'Mean_CO2':Mean_CO2,
                'Load':load,
            
                    }
# Create a DataFrame
        df = pd.DataFrame(data)
        objective_value = value(m.obj)  # Replace 'obj' with your objective name
        production_amount = sum(df['Load'])
#location-based carbon emission calculation method:  (kg CO2e/kWh)
        if location=='QLD1':
            CI_location_based_method=-sum(df['grid_interaction']*(0.73))/production_amount
        if location=='NSW1':
            CI_location_based_method=-sum(df['grid_interaction']*(0.68))/production_amount
        if location=='TAS1':
            CI_location_based_method=-sum(df['grid_interaction']*(0.12))/production_amount
        if location=='VIC1':
            CI_location_based_method=-sum(df['grid_interaction']*(0.79))/production_amount
        if location=='SA1':
            CI_location_based_method=-sum(df['grid_interaction']*(0.25))/production_amount
#save key indicators:
        purchase_amount=sum(df['grid_pout'])
        sell_amount=sum(df['grid_pin'])
        RE_generation=sum(df['CP_wind']+df['CP_pv'])
        Supply_proportion=(RE_generation-sell_amount)/sum(df['el_pin'])   #which shows the power supply comes from which source
        market_based_emission_TEC=(sum(df['el_pin'])*(1-0.188)-(RE_generation))*0.81/production_amount
        market_based_emission=(-1*purchase_amount*(1-0.188)-(sell_amount))*0.81/production_amount
        LGCs=(-1*purchase_amount*(1-0.188)-(sell_amount))/1000
        LCOH = value(m.LCOH)/production_amount+value(m.el_VOM)
        LCOH_LGCs = (value(m.LCOH)+LGCs*25)/production_amount+value(m.el_VOM)
        capex=value(m.capex)
        grid_cost = value(m.grid_cost)
        production_amount = sum(df['Load'])
        MEF_carbon_emissions = sum(df['CO2']) / value(m.production_amount)
        MeanEI_carbon_emissions= sum(df['Mean_CO2']) / value(m.production_amount)
        h2_initial_storage_level = value(m.initial_h2_storage_value)
        wind_capacity=value(m.wind_capacity)
        pv_capacity=value(m.pv_capacity)
        electrolyser_capacity=value(m.electrolyser_max_input)
        hydrogen_storage_capacity=value(m.h2_storage_capacity)
        max_power_export=value(m.max_power_export)
        
        print("Simultaneity_obligation_interval:",num_interval)
        print("LCOH:",value(m.LCOH)/value(m.production_amount)+value(m.el_VOM))
        print("Capex:",capex)
        print('Model running resolution:',step)
        print("grid_cost:",value(m.grid_cost)/value(m.production_amount))
        print("production_amount:",sum(df['Load']))
        print('EI_TEC:',market_based_emission_TEC)
        print("EI_MEF:",sum(df['CO2'])/value(m.production_amount))
        print("EI_Mean:",sum(df['Mean_CO2'])/value(m.production_amount))
        print("EI_L:",CI_location_based_method)
        print("EI_GCO:",market_based_emission)  
        print("Grid_connection_fee",value(m.grid_connection_fee))
        print("H2_initial_storage_level:", value(m.initial_h2_storage_value), 
              "\nH2_final_storage_level:", value(m.h2_storage_level[end_index]))


        
# Create a dictionary with key indicators
        indicators = {
                'Location':location,
                'Year':year,
                'LCOH': LCOH,
                'LCOH_LGCs':LCOH_LGCs,
                'Capex':capex,
                'Step':step,
                'RE_supply_proportion':Supply_proportion,
                #'Sell_proportion':sell_amount/RE_generation,
                'EI_TEC':market_based_emission_TEC,
                'EI_GCO': market_based_emission,
                'EI_MEF': MEF_carbon_emissions,
                'EI_MeanEI': MeanEI_carbon_emissions,
                'EI_L':CI_location_based_method,
                'LGCs':LGCs,
                'wind_capacity':wind_capacity,
                'pv_capacity':pv_capacity,
                'electrolyser_capacity':electrolyser_capacity,
                'hydrogen_storage_capacity':hydrogen_storage_capacity,
                'Grid_max_power_export':max_power_export,
                'grid_cost': grid_cost,
                'production_amount': production_amount,
                'H2_initial_storage_level': h2_initial_storage_level,
                'Simultaneity_obligation_interval': num_interval,
                'ratio':ratio,
                'Simultaneity_obligation':SO_k,
                'Batch_interval':batch_interval,
                'Supply_ratio':supply_ratio,
                    }

            # Create a DataFrame
        indicators_df = pd.DataFrame([indicators])
        return df,indicators_df   
    else:
        print("No optimal solution found")
        print("Solver Status:", results.solver.status)
        
        return None, None

    

In [54]:
Simultaneity_obligation=[0, 1,24,24*7,24*30,24*30*3,24*30*6,8759]
Ratio = np.arange(0,1.1, 0.2).tolist()
df=pd.DataFrame()

In [50]:
operation_result,key_indicators=ECHO(year=2021,location='QLD1',step=30,grid=1,supply_ratio=0,source_df=source,num_interval=0,ratio=0.4,SO_k=0,batch_interval=1)
df=df.append(key_indicators, ignore_index=True)

All values are reassigned
All values are reassigned
Number of batch intervals: 8760
On-Grid
Simultaneity_obligation is off
supply_ratio: 0 No restriction on selling amount
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 2.9740507392595488
Capex: 4447735864.049974
Model running resolution: 30
grid_cost: 0.148135469770051
production_amount: 157671000.0
EI_TEC: 13.711255640143145
EI_MEF: 18.489187824569154
EI_Mean: 19.342262393788953
EI_L: 20.324819837942055
EI_GCO: 17.83637001635088
Grid_connection_fee 120665566.88997439
H2_initial_storage_level: 0.0 
H2_final_storage_level: 0.0


15
30
60


In [56]:
for j in [15,30,60]:
    for i in Ratio:
        operation_result,key_indicators=ECHO(year=2021,location='QLD1',step=j,grid=1,supply_ratio=0,source_df=source,num_interval=0,ratio=i,SO_k=0,batch_interval=1)
        df=df.append(key_indicators, ignore_index=True)


All values are reassigned
All values are reassigned
Number of batch intervals: 8760
On-Grid
Simultaneity_obligation is off
supply_ratio: 0 No restriction on selling amount
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.2364056343157963
Capex: 2111898337.76
Model running resolution: 15
grid_cost: 2.5306542066021094
production_amount: 157666500.0
EI_TEC: 38.18534400000086
EI_MEF: 37.71055018191915
EI_Mean: 40.343317212505596
EI_L: 42.381714285699594
EI_GCO: 38.18534400000086
Grid_connection_fee 67818777.5
H2_initial_storage_level: 0.0 
H2_final_storage_level: 0.0
All values are reassigned
All values are reassigned
Number of batch intervals: 8760
On-Grid
Simultaneity_obligation is off
supply_ratio: 0 No restriction on selling amount
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.105335751179512
Capex: 3024329357.51295
Model running resolution: 15
grid_cost: 1.5548614011906996
production_amount: 157666500.0
EI_TEC: 28.395504945289172
EI_MEF: 30.07395

In [66]:
Proportion=np.arange(0.01,0.3,0.02)
Proportion

array([0.01, 0.03, 0.05, 0.07, 0.09, 0.11, 0.13, 0.15, 0.17, 0.19, 0.21,
       0.23, 0.25, 0.27, 0.29])

In [36]:
path=r'D:\Do it\Phd\ECHO\echo-2.0.1\src\States\result\QLD\Result record\Batch interval\Batch interval results comparison 2021.csv'
df.to_csv(path)

In [26]:
for j in [60,30,15]:
    for i in Ratio:
        operation_result,key_indicators=ECHO(2022,'QLD1',j,0,source_df,num_interval=0,ratio=i,SO_k=0,batch_interval=1)
        df=df.append(key_indicators, ignore_index=True)

All values are reassigned
All values are reassigned
Number of batch intervals: 8760
Simultaneity_obligation is off
supply_ratio: 0 No restriction on selling amount
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 8.511914531892023
Model running resolution: 60
grid_cost: 6.808002410086309
production_amount: 157680000.0
EI_TEC: 38.185343999999816
EI_MEF: 33.993081356970414
EI_Mean: 38.84268380787753
EI_L: 42.38171428571911
EI_GCO: 38.185343999999816
Grid_connection_fee 402537260.0
H2_initial_storage_level: 1767723.380905506 
H2_final_storage_level: 1767723.380905506
All values are reassigned
All values are reassigned
Number of batch intervals: 8760
Simultaneity_obligation is off
supply_ratio: 0 No restriction on selling amount
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 6.584296197047663
Model running resolution: 60
grid_cost: 4.359369929521019
production_amount: 157680000.0
EI_TEC: 28.39611604271401
EI_MEF: 27.4857709872797
EI_Mean: 30.70238298909681

In [28]:
path=r'D:\Do it\Phd\ECHO\echo-2.0.1\src\States\result\QLD\Result record\Model validation\5 CO2 results comparison 2021-2022.csv'
df.to_csv(path)

In [27]:
df

,Location,Year,LCOH,Step,RE_supply_proportion,EI_TEC,EI_GCO,EI_MEF,EI_MeanEI,EI_L,wind_capacity,pv_capacity,electrolyser_capacity,hydrogen_storage_capacity,Grid_max_power_export,grid_cost,production_amount,H2_initial_storage_level,Simultaneity_obligation_interval,ratio,Simultaneity_obligation,Batch_interval,Supply_ratio
0,QLD1,2021,4.274288,60,0.000000,38.185344,38.185344,37.693965,40.369667,42.381714,0.0,0.0,1750162,10003211,1.750162e+06,4.052969e+08,157680000.0,0.000000e+00,0,0.0,0,1,0
1,QLD1,2021,3.810991,60,0.186141,28.396116,30.041779,30.016979,31.951134,33.559324,403017.8,291088.8,1750162,10003211,1.636025e+06,2.516484e+08,157680000.0,0.000000e+00,0,0.2,0,1,0
2,QLD1,2021,3.353028,60,0.370512,18.606888,21.882565,22.334299,23.543888,24.736933,806035.6,582177.6,1750162,10003211,1.488606e+06,9.943956e+07,157680000.0,0.000000e+00,0,0.4,0,1,0
3,QLD1,2021,2.906971,60,0.552515,8.817660,13.702411,14.615389,15.121858,15.914542,1209053.4,873266.4,1750162,10003211,1.395566e+06,-5.187013e+07,157680000.0,0.000000e+00,0,0.6,0,1,0
4,QLD1,2021,2.532945,60,0.716685,-0.513154,5.823013,7.374715,7.072660,7.505290,1612071.2,1164355.2,1750162,10003211,1.538144e+06,-1.960612e+08,157680000.0,0.000000e+00,0,0.8,0,1,0
5,QLD1,2021,2.260330,60,0.842421,-8.940966,-1.493171,0.906692,-0.221380,-0.090145,2015089.0,1455444.0,1750162,10003211,1.922680e+06,-3.286152e+08,157680000.0,0.000000e+00,0,1.0,0,1,0
6,QLD1,2021,3.526508,30,0.000000,38.185344,38.185344,34.848212,39.215250,42.381714,0.0,0.0,1750162,10003211,1.750162e+06,2.873559e+08,157671000.0,0.000000e+00,0,0.0,0,1,0
7,QLD1,2021,3.148587,30,0.179709,28.395709,29.984506,27.159742,30.859114,33.558957,403017.8,291088.8,1750162,10003211,1.664011e+06,1.466693e+08,157671000.0,0.000000e+00,0,0.2,0,1,0
8,QLD1,2021,2.783150,30,0.355999,18.606073,21.753436,19.551818,22.527771,24.736199,806035.6,582177.6,1750162,10003211,1.563417e+06,8.210920e+06,157671000.0,7.863165e+03,0,0.4,0,1,0
9,QLD1,2021,2.434725,30,0.526281,8.816438,13.469261,11.850198,14.235970,15.913441,1209053.4,873266.4,1750162,10003211,1.443805e+06,-1.272230e+08,157671000.0,1.851012e+04,0,0.6,0,1,0


In [73]:
for j in Simultaneity_obligation:
    operation_result,key_indicators=ECHO(2021,'QLD1',60,0.05,source_df,num_interval=j,ratio=0,SO_k=1,batch_interval=1)
    df=df.append(key_indicators, ignore_index=True)

All values are reassigned
All values are reassigned
Number of batch intervals: 8760
Simultaneity_obligation is off
supply_ratio: 0.05
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 2.4789500220683376
Model running resolution: 60
grid_cost: -1.7145491713753434
production_amount: 157680000.0
EI_TEC: -6.535594985204395
EI_MEF: 1.655203716878622
EI_L: 2.077658163246098
EI_GCO: 1.4515647175530944
Grid_connection_fee 420763823.1443344
H2_initial_storage_level: 12787.146627945189 
H2_final_storage_level: 12787.146627945189
All values are reassigned
All values are reassigned
Number of batch intervals: 8760
Simultaneity_obligation is on
supply_ratio: 0.05
Optimal Solution Found
Simultaneity_obligation_interval: 1
LCOH: 2.574881016005337
Model running resolution: 60
grid_cost: -2.1155728396670157
production_amount: 157680000.0
EI_TEC: -11.316009383458507
EI_MEF: -1.3432374997875651
EI_L: -2.2306165413533687
EI_GCO: -2.475067669172918
Grid_connection_fee 447354500.81978524
H2_in

### Test different batch interval in the model

In [32]:
Batch=[1,24,24*7,24*30,24*30*3,24*30*6,8759]

In [33]:
for j in Ratio:
    for i in Batch:
        operation_result,key_indicators=ECHO(2021,'QLD1',60,0,source_df,num_interval=0,ratio=j,SO_k=0,batch_interval=i)
        df=df.append(key_indicators, ignore_index=True)

All values are reassigned
All values are reassigned
Number of batch intervals: 8760
Simultaneity_obligation is off
supply_ratio: 0 No restriction on selling amount
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 4.27428811556466
Model running resolution: 60
grid_cost: 2.5703759937589443
production_amount: 157680000.0
EI_TEC: 38.18534399999962
EI_MEF: 37.69396518282716
EI_Mean: 40.36966734740072
EI_L: 42.38171428571889
EI_GCO: 38.18534399999962
Grid_connection_fee 402537260.0
H2_initial_storage_level: 0.0 
H2_final_storage_level: 0.0
All values are reassigned
All values are reassigned
Number of batch intervals: 365
Simultaneity_obligation is off
supply_ratio: 0 No restriction on selling amount
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 4.277160401240856
Model running resolution: 60
grid_cost: 2.5732482794351403
production_amount: 157680000.0
EI_TEC: 38.18534399999987
EI_MEF: 37.67341236738419
EI_Mean: 40.363697139258626
EI_L: 42.38171428571919
EI_G

In [34]:
df

,Location,Year,LCOH,Step,RE_supply_proportion,EI_TEC,EI_GCO,EI_MEF,EI_MeanEI,EI_L,wind_capacity,pv_capacity,electrolyser_capacity,hydrogen_storage_capacity,Grid_max_power_export,grid_cost,production_amount,H2_initial_storage_level,Simultaneity_obligation_interval,ratio,Simultaneity_obligation,Batch_interval,Supply_ratio
0,QLD1,2021,4.274288,60,0.000000,38.185344,38.185344,37.693965,40.369667,42.381714,0.0,0.0,1750162,10003211,1.750162e+06,4.052969e+08,157680000.0,0.000000,0,0.0,0,1,0
1,QLD1,2021,4.277160,60,0.000000,38.185344,38.185344,37.673412,40.363697,42.381714,0.0,0.0,1750162,10003211,1.750162e+06,4.057498e+08,157680000.0,0.000000,0,0.0,0,24,0
2,QLD1,2021,4.301900,60,0.000000,38.185344,38.185344,37.601658,40.361493,42.381714,0.0,0.0,1750162,10003211,1.750162e+06,4.096507e+08,157680000.0,120582.027559,0,0.0,0,168,0
3,QLD1,2021,3.810991,60,0.186141,28.396116,30.041779,30.016979,31.951134,33.559324,403017.8,291088.8,1750162,10003211,1.636025e+06,2.516484e+08,157680000.0,0.000000,0,0.2,0,1,0
4,QLD1,2021,3.813831,60,0.186274,28.396116,30.042952,29.997573,31.952935,33.559324,403017.8,291088.8,1750162,10003211,1.637888e+06,2.520626e+08,157680000.0,0.000000,0,0.2,0,24,0
5,QLD1,2021,3.838063,60,0.186465,28.396116,30.044646,29.882717,31.955058,33.559324,403017.8,291088.8,1750162,10003211,1.641411e+06,2.558201e+08,157680000.0,148130.327396,0,0.2,0,168,0
6,QLD1,2021,3.353028,60,0.370512,18.606888,21.882565,22.334299,23.543888,24.736933,806035.6,582177.6,1750162,10003211,1.488606e+06,9.943956e+07,157680000.0,0.000000,0,0.4,0,1,0
7,QLD1,2021,3.355728,60,0.370596,18.606888,21.883301,22.320483,23.543510,24.736933,806035.6,582177.6,1750162,10003211,1.490190e+06,9.983693e+07,157680000.0,0.000000,0,0.4,0,24,0
8,QLD1,2021,3.379125,60,0.371410,18.606888,21.890500,22.199328,23.544390,24.736933,806035.6,582177.6,1750162,10003211,1.497686e+06,1.033912e+08,157680000.0,174406.935373,0,0.4,0,168,0
9,QLD1,2021,2.906971,60,0.552515,8.817660,13.702411,14.615389,15.121858,15.914542,1209053.4,873266.4,1750162,10003211,1.395566e+06,-5.187013e+07,157680000.0,0.000000,0,0.6,0,1,0


### Validation model with different time resolution test

In [17]:
resolution=[15,30,60]


15
30
60


In [27]:
for r in resolution:
    for i in range(2019,2020,1):
        if i == 2020:
            continue
        for j in Ratio:
            operation_result,key_indicators=ECHO(i,'QLD1',r,source_df,num_interval=0,ratio=j,SO_k=0)
            df=df.append(key_indicators, ignore_index=True)

All values are reassigned
All values are reassigned
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.9859659538090524
Model running step: 15
grid_cost: 2.462010609979376
production_amount: 157666500.0
EI_TEC: 38.18534399999973
EI_MEF: 42.25069553135366
EI_L: 42.38171428571903
EI_GCO: 38.18534399999973
H2_initial_storage_level: 9814211.0 
H2_final_storage_level: 9814211.0
All values are reassigned
All values are reassigned
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.7920448061583647
Model running step: 15
grid_cost: 2.0059873069862193
production_amount: 157666500.0
EI_TEC: 33.290424472644034
EI_MEF: 38.3566057478279
EI_L: 37.97024360056488
EI_GCO: 33.59976163992436
H2_initial_storage_level: 9820490.410474444 
H2_final_storage_level: 9820490.410474444
All values are reassigned
All values are reassigned
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interva

In [8]:
for j in Ratio:
    operation_result,key_indicators=ECHO(2021,'QLD1',60,0,source_df,num_interval=0,ratio=j,SO_k=0)
    df=df.append(key_indicators, ignore_index=True)

All values are reassigned
All values are reassigned
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 4.094207283192745
Model running resolution: 60
grid_cost: 2.5703759937589443
production_amount: 157680000.0
EI_TEC: 38.18534399999962
EI_MEF: 37.69396518282716
EI_L: 42.38171428571889
EI_GCO: 38.18534399999962
H2_initial_storage_level: 0.0 
H2_final_storage_level: 0.0
All values are reassigned
All values are reassigned
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.8250448815805247
Model running resolution: 60
grid_cost: 2.039133877057281
production_amount: 157680000.0
EI_TEC: 33.290730021356765
EI_MEF: 33.94040189680925
EI_L: 37.97051897162817
EI_GCO: 33.979985389721925
H2_initial_storage_level: 0.0 
H2_final_storage_level: 0.0
All values are reassigned
All values are reassigned
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.55624312254211

In [16]:
for j in Ratio:
    for i in range(2019,2023,1):
        if i == 2020:
            continue
        CI=carbon_intensity(i,'QLD1')
        Price=price(i,'QLD1')
        operation_result,key_indicators=ECHO(i,'QLD1',data,num_interval=0,ratio=j,SO_k=0)
        df=df.append(key_indicators, ignore_index=True)

Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.985873234904276
grid_cost: 2.4619178910745996
production_amount: 157666500
grid_carbon_emissions (MEF): 42.24939496782242
grid_carbon_emissions (Local): 42.38050465834552
grid_carbon_emissions (Market): 38.184254142306436
H2_initial_storage_level: 9818711.0 
H2_final_storage_level: 9814211.0
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 2.7190729131653075
grid_cost: 1.1951175693356308
production_amount: 157666500
grid_carbon_emissions (MEF): 28.016926064873672
grid_carbon_emissions (Local): 42.38050465834562
grid_carbon_emissions (Market): 38.184254142306514
H2_initial_storage_level: 1074524.30610236 
H2_final_storage_level: 1070024.30610236
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 4.007945897588735
grid_cost: 2.483990553759059
production_amount: 157666500
grid_carbon_emissions (MEF): 2

In [13]:
for j in Simultaneity_obligation:
    for i in range(2013,2023,1):
        if i == 2020:
            continue
        CI=carbon_intensity(i,'QLD1')
        Price=price(i,'QLD1')
        operation_result,key_indicators=ECHO(i,'QLD1',source_df,num_interval=j,ratio=0,SO_k=1)
        df=df.append(key_indicators, ignore_index=True)

Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.856358001613762
grid_cost: 0.4207081660747596
production_amount: 157680000
grid_carbon_emissions (MEF): 9.493318196331979
grid_carbon_emissions (Local): 8.41116314026276
grid_carbon_emissions (Market): 6.4109944646765555
H2_initial_storage_level: 0.0 
H2_final_storage_level: 0.0
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.5251571286123595
grid_cost: 0.2189997013067592
production_amount: 157680000
grid_carbon_emissions (MEF): 11.113567767641864
grid_carbon_emissions (Local): 10.45567513791517
grid_carbon_emissions (Market): 8.079715066510179
H2_initial_storage_level: 4538166.748167119 
H2_final_storage_level: 4538166.748167119
Simultaneity_obligation is off
Optimal Solution Found
Simultaneity_obligation_interval: 0
LCOH: 3.517568284157517
grid_cost: 0.9048984500769469
production_amount: 157680000
grid_carbon_emissions (MEF): 26.091274713

In [9]:
df

,Location,Year,LCOH,Step,RE_supply_proportion,EI_TEC,EI_GCO,EI_MEF,EI_L,wind_capacity,pv_capacity,electrolyser_capacity,hydrogen_storage_capacity,grid_cost,production_amount,H2_initial_storage_level,Simultaneity_obligation_interval,ratio,Simultaneity_obligation
0,QLD1,2021,4.094207,60,0.000000,38.185344,38.185344,37.693965,42.381714,0.0,0.0,1750162,10003211,4.052969e+08,157680000.0,0.0,0,0.0,0
1,QLD1,2021,3.825045,60,0.077962,33.290730,33.979985,33.940402,37.970519,201508.9,145544.4,1750162,10003211,3.215306e+08,157680000.0,0.0,0,0.1,0
2,QLD1,2021,3.556243,60,0.155068,28.396116,29.767059,30.171458,33.559324,403017.8,291088.8,1750162,10003211,2.378212e+08,157680000.0,0.0,0,0.2,0
3,QLD1,2021,3.287792,60,0.231665,23.501502,25.549639,26.440654,29.148128,604526.7,436633.2,1750162,10003211,1.541671e+08,157680000.0,0.0,0,0.3,0
4,QLD1,2021,3.019702,60,0.307680,18.606888,21.327069,22.723481,24.736933,806035.6,582177.6,1750162,10003211,7.056995e+07,157680000.0,0.0,0,0.4,0
5,QLD1,2021,2.752064,60,0.382149,13.712274,17.090828,18.987662,20.325738,1007544.5,727722.0,1750162,10003211,-1.295587e+07,157680000.0,0.0,0,0.5,0
6,QLD1,2021,2.484888,60,0.454919,8.817660,12.839574,15.252322,15.914542,1209053.4,873266.4,1750162,10003211,-9.640889e+07,157680000.0,0.0,0,0.6,0
7,QLD1,2021,2.219789,60,0.523635,3.923046,8.552469,11.523845,11.503347,1410562.3,1018810.8,1750162,10003211,-1.795345e+08,157680000.0,0.0,0,0.7,0
8,QLD1,2021,1.958588,60,0.585387,-0.971568,4.203807,7.830647,7.092152,1612071.2,1164355.2,1750162,10003211,-2.620454e+08,157680000.0,0.0,0,0.8,0
9,QLD1,2021,1.700863,60,0.639033,-5.866182,-0.216528,4.113388,2.680956,1813580.1,1309899.6,1750162,10003211,-3.440082e+08,157680000.0,0.0,0,0.9,0


In [17]:
path=r'D:\Do it\Phd\ECHO\echo-2.0.1\src\States\result\QLD\Result record\2021 RE_supply_proportion (different ratios).csv'
df.to_csv(path)

##### Plot